### Open API를 활용한 공공 데이터 수집
- 공공 데이터
    - 공공기관이 생성한 데이터
    - 누구나 무료로 자유롭게 활용 가능
 
- 공공데이터 제공 사이트 : www.data.go.kr
    - 회원가입후 데이터 사용 신청하고 servicekey받아서 사용

#### 국립 중앙의료원 제공 전국 약국 정보데이터 
- 요청변수가 코드가 아닌 실제 지역명(한글 이름) 사용해야함
- 한글 인코딩 처리 : quote()
- 요청 주소
endpoint= 'http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyListInfoInqire?'

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [4]:
from urllib.request import quote     # 한글 인코딩 처리
import pandas as pd
import requests
import bs4

In [5]:
# 요청 주소
endpoint= 'http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyListInfoInqire?'

In [6]:
# 내 serviceKey
serviceKey = 'fldUxTB3BQ%2BwSxomuOPnk9CeYhO4NbV5JvRq3U1YrWRqpFFz%2FKQABwSSsTR%2FhIgVJZrqUvds9Q9f9Fw0iwUT7g%3D%3D'

In [7]:
# parameter 변수 설정
QT = "1"   # 월요일에 여는 약국
QN = quote('삼성약국') # 기관명(약국이름) 인코딩 -> '삼성약국'이 포함된 모든 약국 반환
Q0 = quote('서울특별시')  # 지역명(시-도)
Q1 = quote('강남구')    # 시군구
ORD = 'NAME'  # order 기준: 기관명
pageNo = '1'
numOfRows = '10'   # 목록 건수 

In [9]:
# 파라미터 설정
params = 'serviceKey='+ serviceKey +\
        '&Q0=' + Q0 +\
        '&Q1=' + Q1 +\
        '&QN=' + QN +\
        '&QT=' + QT +\
        '&ORD=' + ORD +\
        '&pageNo=' + pageNo +\
        '&numOfRows=' + numOfRows

In [10]:
# 요청 url
url = endpoint + params

In [12]:
url

'http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyListInfoInqire?serviceKey=fldUxTB3BQ%2BwSxomuOPnk9CeYhO4NbV5JvRq3U1YrWRqpFFz%2FKQABwSSsTR%2FhIgVJZrqUvds9Q9f9Fw0iwUT7g%3D%3D&Q0=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&Q1=%EA%B0%95%EB%82%A8%EA%B5%AC&QN=%EC%82%BC%EC%84%B1%EC%95%BD%EA%B5%AD&QT=1&ORD=NAME&pageNo=1&numOfRows=10'

In [18]:
# service requests 및 result parsing
result = requests.get(url)    # 성공시 200
bs_obj = bs4.BeautifulSoup(result.content, "html.parser")

In [20]:
# item 추출 
items = bs_obj.findAll('item')
len(items)

4

In [28]:
# 데이터 추출
# 반환된 결과에서 약국이름 <dutyname>, 주소 추출<dutyaddr>
dutyname = [duty.text for duty in bs_obj.findAll('dutyname')]
dutyaddr = [duty.text for duty in bs_obj.findAll('dutyaddr')]
dutyname
dutyaddr

['E-삼성약국', '무지개삼성약국', '삼성약국', '이삼성약국']

['서울특별시 강남구 일원로 53 (일원동) ',
 '서울특별시 강남구 일원로 45, 대훈빌딩 1층 (일원동)',
 '서울특별시 강남구 광평로19길 10, 101호 (수서동, 까치마을아파트상가)',
 '서울특별시 강남구 일원로 53, 1층 (일원동, 소림빌딩)']

### 서울의 모든 약국 

In [35]:
# parameter 변수 설정
QT = "1"   # 월요일에 여는 약국
QN = quote('삼성약국') # 기관명(약국이름) 인코딩 -> '삼성약국'이 포함된 모든 약국 반환
Q0 = quote('서울특별시')  # 지역명(시-도)
Q1 = quote('강남구')    # 시군구
ORD = 'NAME'  # order 기준: 기관명
pageNo = '1'
numOfRows = '50'   # 목록 건수 

In [36]:
# 파라미터 설정
params = 'serviceKey='+ serviceKey +\
        '&Q0=' + Q0 +\
        '&ORD=' + ORD +\
        '&numOfRows=' + numOfRows

In [37]:
# 요청 Url
url = endpoint + params
url

'http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyListInfoInqire?serviceKey=fldUxTB3BQ%2BwSxomuOPnk9CeYhO4NbV5JvRq3U1YrWRqpFFz%2FKQABwSSsTR%2FhIgVJZrqUvds9Q9f9Fw0iwUT7g%3D%3D&Q0=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C&ORD=NAME&numOfRows=50'

In [71]:
# service requests 및 result parsing
result = requests.get(url)    # 성공시 200
bs_obj = bs4.BeautifulSoup(result.content, "html.parser")
# bs_obj

In [43]:
items = bs_obj.findAll('item')
len(items)
items[1].find('')

50

In [54]:
# 서울시 약국 정보(약국명, 전화, 주소)추출해서 데이터프레임 생성
# 약국명 : dutyname
# 전화 : dutytel1
# 주소: dutyaddr
dutyaddr, dutyname, dutytel = [[] for _ in range(3)]

for item in bs_obj.findAll('item'):
    dutyname.append(item.find('dutyname').text)
    dutytel.append(item.find('dutytel1').text)
    dutyaddr.append(item.find('dutyaddr').text)

duty_df = pd.DataFrame({'약국명':dutyname, '전화':dutytel, '주소':dutyaddr})
duty_df

,약국명,전화,주소
0,100세건강약국,02-445-1460,"서울특별시 강남구 헌릉로571길 7, 강남레체 1층 101호 (세곡동)"
1,100세약국,02-2678-8217,"서울특별시 영등포구 양평로 24, (당산동6가)"
2,13층약국,070-7718-1316,"서울특별시 중구 서소문로 116, 유원빌딩 1304호 (서소문동)"
3,1898광장약국,02-0000-0000,"서울특별시 중구 명동길 74, 카톨릭회관 신관 B113호 (명동2가)"
4,1번약국,02-872-7583,서울특별시 관악구 남부순환로 1832 1층
5,1번약국,02-851-1155,서울특별시 구로구 구로동로 132 (구로동)
6,1번출구새싹약국,02-919-7167,"서울특별시 노원구 화랑로 337, 102호 (월계동)"
7,1층메디컬약국,02-891-1112,"서울특별시 금천구 시흥대로 214, 비즈메드빌딩 1층 102호 (시흥동)"
8,1층엔약국,02-518-3014,"서울특별시 강남구 강남대로 624, ICT TOWER 1층 일부호 (신사동)"
9,1층천호약국,02-478-1178,"서울특별시 강동구 천호대로 1006, 브라운스톤천호 105호 (성내동)"


In [69]:
# # 다른 방법
# dutyname = [item.select_one('dutyname').text for item in bs_obj.findAll('items')]
# dutyaddr = [item.select_one('dutyaddr').text for item in bs_obj.findAll('items')]
# dutytel = [item.select_one('dutytel1').text for item in bs_obj.findAll('items')]


# duty_df = pd.DataFrame({'약국명':dutyname, '전화':dutytel, '주소':dutyaddr})
# duty_df

In [73]:
# 월요일 밤 9시 이후까지 하는 약국 추출 : 2100 
# 월요일 영업 마감시간 <dutytime1c>
# df 저장
dutyaddr, dutyname, dutytel = [[] for _ in range(3)]

for item in bs_obj.findAll('item'):
    if int(item.find('dutytime1c').text) >= 2100:
        dutyname.append(item.find('dutyname').text)
        dutytel.append(item.find('dutytel1').text)
        dutyaddr.append(item.find('dutyaddr').text)

duty_df = pd.DataFrame({'약국명':dutyname, '전화':dutytel, '주소':dutyaddr})
duty_df

,약국명,전화,주소
0,21세기메디칼약국,02-2252-6626,서울특별시 성동구 무수막길 97 (금호동2가)
1,21세기신천지약국,02-2678-3222,서울특별시 영등포구 영등포로 232-1
2,21세기약국,02-2678-2854,서울특별시 영등포구 여의대방로 177 (신길동)
3,365 새누리약국,02-2648-1858,"서울특별시 양천구 등촌로 32, 목동태영프라자 102호 (목동)"
4,365강남역약국,02-554-5628,"서울특별시 강남구 강남대로 지하396, 강남역 103-104호 (역삼동)"
5,365구생약국,02-353-1144,"서울특별시 은평구 통일로 739, 1층 (대조동)"
6,365그랜드약국,02-896-1223,"서울특별시 금천구 시흥대로62길 29, 1층 (시흥동)"
7,365그린약국,02-2602-8006,"서울특별시 강서구 강서로 43, B104호 (화곡동, 도양라비앙타워)"
8,365마트약국,02-2633-4865,"서울특별시 영등포구 영중로 125, 롯데마트 1층 (영등포동8가)"
9,365미소약국,02-387-1123,"서울특별시 은평구 진관3로 11, 1층 (진관동, 은평뉴타운 힐데스하임)"


In [67]:
int(bs_obj.findAll('items')[0].find('dutytime1c').text) >= 2100

False